In [2]:
import pandas as pd; import numpy as np
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split
import pickle

In [ ]:
#Step 0: Load
ifile = './data/pets1k.csv'
df = pd.read_csv(ifile,header=0)
#Step 1: Prepare the data
#Features we want: Price, Popularity, Age, Personality, Size
trimdf = pd.DataFrame()
trimdf['price'] = df['price']
trimdf['age'] = df['age']
trimdf['personality]'] = df['personality']
trimdf['size'] = df['size']
trimdf